In [14]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import os
import librosa
import soundfile
import keras

In [20]:
RANDOM = np.random.RandomState(87)


In [28]:
my_dir = '/home/birdsong/train'
os.mkdir(my_dir)
for i in range(0,24):
    os.mkdir(my_dir+'/s'+str(i))







In [33]:
#load train_tp.csv into a pandas df
import random
df = pd.read_csv('/home/birdsong/kaggle-data/train_tp.csv')
display(df)
#loop through each row of the df (supposedly inefficient, but seems necessary here)
for index, row in df.iterrows():
    infile =  '/home/birdsong/kaggle-data/train/%s.flac' %row['recording_id']
    outfile = '/home/birdsong/train/s%d/%d.flac' %(row['species_id'],index) 
    spectograms = ''
    sig, sr = librosa.load(infile)
    tmin = row['t_min']*sr 
    tmax = row['t_max']*sr
    l = len(sig)
    if tmax - tmin < 3*sr:
        if  tmax - 3*sr > 0:
            upper = np.random.randint(tmax, min(tmin + 3*sr,l)) 
            lower = upper - 3*sr
        else:
            lower = np.random.randint(0, tmin)
            upper = lower + 3*sr
    else:
        lower = round(tmin + (tmax - tmin - 3*sr)/2)
        upper = round(tmax - (tmax - tmin - 3*sr)/2)
    
    sig = sig[lower:upper]
    
    # uncomment to actually write files
    soundfile.write(outfile, sig, 22050)


,recording_id,species_id,songtype_id,t_min,f_min,t_max,f_max
0,003bec244,14,1,44.5440,2531.250,45.1307,5531.25
1,006ab765f,23,1,39.9615,7235.160,46.0452,11283.40
2,007f87ba2,12,1,39.1360,562.500,42.2720,3281.25
3,0099c367b,17,4,51.4206,1464.260,55.1996,4565.04
4,009b760e6,10,1,50.0854,947.461,52.5293,10852.70
...,...,...,...,...,...,...,...
1211,fe8d9ac40,13,1,53.4720,93.750,54.0960,843.75
1212,fea6b438a,4,1,43.5787,2531.250,45.7653,4031.25
1213,ff2eb9ce5,0,1,15.2267,5906.250,16.0213,8250.00
1214,ffb8d8391,5,1,14.3467,4781.250,16.6987,10406.20


-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150
-66150

In [ ]:
!pwd
!mkdir classifier

In [ ]:
!python3 /home/birdsong/BirdNET-Analyzer/train.py --i /home/birdsong/train --o /home/birdsong/classifier/custom-classifier.tflite

In [81]:
audio_path = 'kaggle-data/test/000316da7.flac'
sig, _ = librosa.load(audio_path, 
            sr=48000, mono=True, res_type='kaiser_fast')
chunk_length = 3*48000
nchunks = len(sig) // chunk_length
chunks = sig[:nchunks*chunk_length].reshape(nchunks, chunk_length)

In [82]:
dirname = 'clips/s03-1'

chunks = []

for filename in os.listdir(dirname):
    sig, _ = librosa.load(dirname + '/' + filename, sr = 48000, res_type = 'kaiser_fast')
        
    sig.resize(3*48000)
    sig[len(sig):3*48000] = sig[0:3*48000 - len(sig)]
    chunks.append(sig)

In [83]:
#load BirdNET TFLITE model
interpreter = tf.lite.Interpreter('/home/birdsong/BirdNET-Analyzer/checkpoints/V2.3/BirdNET_GLOBAL_3K_V2.3_Model_FP32.tflite')
interpreter.allocate_tensors()

        # Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

        # Get input tensor index
INPUT_LAYER_INDEX = input_details[0]["index"]

OUTPUT_LAYER_INDEX = output_details[0]["index"] - 1





In [84]:
#load custom model
model_path = '/home/birdsong/classifier/custom-classifier.tflite'
custom_interpreter = tf.lite.Interpreter(model_path)
custom_interpreter.allocate_tensors()


input_details = custom_interpreter.get_input_details()
output_details = custom_interpreter.get_output_details()

# Get input tensor index
C_INPUT_LAYER_INDEX = input_details[0]["index"]

# Get classification output
C_OUTPUT_LAYER_INDEX = output_details[0]["index"]

In [85]:
#prediction
#Get Feature Vector from Penultimate layer to feed into custom model. 

#Reshape input tensor?
interpreter.resize_tensor_input(INPUT_LAYER_INDEX, [len(chunks), *chunks[0].shape])
interpreter.allocate_tensors()

# Extract feature embeddings
interpreter.set_tensor(INPUT_LAYER_INDEX, np.array(chunks, dtype="float32"))
interpreter.invoke()
feature_vector = interpreter.get_tensor(OUTPUT_LAYER_INDEX)


In [86]:
custom_interpreter.resize_tensor_input(C_INPUT_LAYER_INDEX, [len(feature_vector), *feature_vector[0].shape])
custom_interpreter.allocate_tensors()
custom_interpreter.set_tensor(C_INPUT_LAYER_INDEX, np.array(feature_vector, dtype="float32"))
custom_interpreter.invoke()
prediction = custom_interpreter.get_tensor(C_OUTPUT_LAYER_INDEX)


In [56]:
def flat_sigmoid(x, sensitivity=-1):
    return 1 / (1.0 + np.exp(sensitivity * np.clip(x, -15, 15)))

label_path = '/home/birdsong/classifier/custom-classifier_Labels.txt'
with open(label_path) as f:
    labels = [l.replace('\n', '') for l in f.readlines()]
    


In [95]:
A = (5,)
B = [*A]
C = [A]
B
C

[(5,)]

In [64]:
prediction.shape

(50, 24)

In [78]:
for i in range(50):
    print([(j,prediction[i,j],prediction[i,3]) for j in range(24) if prediction[i,j] == max(prediction[i,:])])

[(17, -6.821684, -11.315914)]
[(17, -5.809486, -14.933146)]
[(17, -2.980143, -22.735565)]
[(17, -3.6949565, -18.562466)]
[(4, -6.3722973, -19.251535)]
[(7, -4.2951, -13.657893)]
[(16, 4.094361, -13.601914)]
[(9, -7.216944, -14.6803465)]
[(17, -3.0471718, -18.728035)]
[(9, 0.6261329, -14.270103)]
[(7, -4.508502, -19.059704)]
[(12, -5.8623524, -19.696115)]
[(17, -3.8041904, -17.295801)]
[(17, -7.381732, -18.811062)]
[(17, -1.1703887, -10.142771)]
[(4, -4.447891, -16.64745)]
[(4, -3.0166416, -15.506189)]
[(17, -3.4552643, -17.370722)]
[(17, -5.1890464, -18.797533)]
[(4, -4.206142, -13.774076)]
[(17, 1.3215318, -12.849528)]
[(7, -6.949502, -10.911119)]
[(17, -5.2271104, -15.527296)]
[(17, -4.546286, -15.4771)]
[(17, -2.837637, -14.0206)]
[(9, -2.7685738, -15.533041)]
[(4, -6.5191956, -16.25487)]
[(23, -4.679637, -13.709007)]
[(17, -2.6823869, -13.350037)]
[(4, -4.584959, -20.6268)]
[(17, -0.23531085, -18.906044)]
[(17, -4.251354, -17.519417)]
[(17, -4.1573186, -14.426578)]
[(17, 0.7368368,

array([[-15.063241 , -16.874573 , -11.946267 , ...,  -7.750904 ,
        -14.231513 ,  -7.1961713],
       [-17.327015 , -21.247118 , -10.919204 , ..., -13.707156 ,
        -18.53609  , -11.505177 ],
       [-17.999392 , -17.160881 , -13.616973 , ..., -13.660581 ,
        -12.169031 ,  -6.410464 ],
       ...,
       [-13.966583 , -20.150015 , -12.387974 , ..., -10.833371 ,
        -12.839296 ,  -9.04515  ],
       [-18.35683  , -17.633862 , -17.099771 , ..., -14.237976 ,
        -11.520317 , -12.344553 ],
       [-13.849258 , -22.386234 , -13.566904 , ..., -12.914735 ,
        -11.154727 ,  -8.695882 ]], dtype=float32)

In [ ]:
for i in range(50):
    print(prediction[])